<a href="https://colab.research.google.com/github/Arun-KR/GenAI/blob/main/TinyLlama_1_1B_Chat_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing all the dependencies**

In [ ]:
!pip install llama_index
%pip install llama-index-llms-huggingface
%pip install sentence_transformer
%pip install transformers einops accelerate  bitsandbytes
%pip install llama-index-embeddings-huggingface



ERROR: Could not find a version that satisfies the requirement sentence_transformer (from versions: none)
ERROR: No matching distribution found for sentence_transformer


**Load the Document**

In [ ]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('/content/sample_data/data/').load_data()

**Load PromptTemplate Class and Define System Prompt**

In [ ]:
from llama_index.core import PromptTemplate


system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

**Authenticate to Hugging Face Hub**

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

**Initialize Llama Language Model with Configuration**

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
import torch


llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.5, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


**Create Vector Index from Documents with Service Context**

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
embed_model=HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")
Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

**Create Vector Index from Documents with Service Context**

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

**Create a Query Engine**

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response = query_engine.query("provide the summary of the paper")
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (2515 > 2048). Running this sequence through the model will result in indexing errors


mentienticular.STPFFERs andfatt <standard forppings:sent:&# design:1%nbspdingcket\.nbspister <specfattidingfattfatt <7.<<paperfattpped\<<youricallyferently.<based.<pdf <dataents2fatticuliculums.<<gaspecumsided:@<automientientfatticuluefattdingdingding̀----****


In [ ]:
res = query_engine.query("What solutions are proposed for enabling reverse conduction capability in GaN power HEMTs, given the absence of a body diode?")
print(res)


The solutions proposed for enabling reverse conduction capability in GaN power HEMTs are:
1. Viable solution for RC-HEMT, which is suitable for the p-GaN gate HEMT platform.
2. GaN Reverse-Blocking or Bidirectional Transistor, which is used for bidirectional power switches in USB chargers and ac matrix converters.
3. Bidirectional switches, which can be formed by two pairs of reverse-blocking switches.
4. Bi-HEMT, which can be formed by a dual-gate HEMT.


**Prompt Engineering**

In [ ]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
     "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, if the answer doesnt exist dont give on your own , mention out of context"
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: ")
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)
refine_template = PromptTemplate(refine_template_str)

print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    ).query("What solutions are proposed for enabling reverse conduction capability in GaN power HEMTs, given the absence of a body diode.")
)


Reverse-blocking or bidirectional transistors are proposed as solutions for enabling reverse conduction capability in GaN power HEMTs. These transistors share the drift region of the device for both conduction and blocking, providing protection against high voltages in both directions. For example, a bidirectional switch can be formed by two pairs of reverse-blocking switches. The reverse-blocking HEMT (RB-HEMT) can be formed by device-level integration of transistors and diodes, resulting in a reverse-blocking HEMT (RB-HEMT) (166), (167), (168), (169), (170), (171), (172), (173). The bidirectional switch can also be formed by a dual-gate HEMT (166), (167), (168), (169), (174), (175), (176). These devices can also be used for future power switches, as shown in Fig. 18. The reverse-blocking HEMT (RB-HEMT)
